# Basic EM workflow 2 (Restaurants data set)

# Introduction

This IPython notebook explains a basic workflow two tables using *py_entitymatching*. Our goal is to come up with a workflow to match restaurants from Fodors and Zagat sites. Specifically, we want to achieve precision and recall above 96%. The datasets contain information about the restaurants.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.1.3
magellan version: 0.3.2


Matching two tables typically consists of the following three steps:

** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# Read input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [3]:
# Get the paths
path_A = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/fodors.csv'
path_B = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/zagats.csv'

In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 533
Number of tuples in B: 331
Number of tuples in A X B (i.e the cartesian product): 176423


In [6]:
A.head(2)

,id,name,addr,city,phone,type
0,534,arnie mortons of chicago,435 s. la cienega blv.,los angeles,310/246-1501,american
1,535,arts delicatessen,12224 ventura blvd.,studio city,818/762-1221,american


In [7]:
B.head(2)

,id,name,addr,city,phone,type
0,1,apple pan the,10801 w. pico blvd.,west la,310-475-3585,american
1,2,asahi ramen,2027 sawtelle blvd.,west la,310-479-2231,noodle shops


In [8]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

In [9]:
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 200, 1, show_progress=False)
len(A1), len(B1)

# But for the purposes of this notebook, we will use the entire table A and B

(129, 200)

# Block Tables To Get Candidate Set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with no overlap between the names will not match. So we decide the apply blocking over names:

In [10]:
# Blocking plan

# A, B -- Overlap blocker [name] --------------------|---> candidate set

In [11]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C = ob.block_tables(A, B, 'name', 'name', 
                    l_output_attrs=['name', 'addr', 'city', 'phone'], 
                    r_output_attrs=['name', 'addr', 'city', 'phone'],
                    overlap_size=1, show_progress=False
                    )
len(C)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_stringsimjoin\utils\converter.py:99: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if col_type == pd.np.object:
c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_stringsimjoin\utils\validation.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if attr_type != pd.np.object:


2915

# Match tuple pairs in candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following four steps:

1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [12]:
# Sample  candidate set
S = em.sample_table(C, 450)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\sampler\single_table.py:103: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  sample_indices = pd.np.random.choice(len(table), sample_size,


Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [13]:
# Label S
G = em.label_table(S, 'gold')

Column name (gold) is not present in dataframe


In [14]:
len(G)

450

In [15]:
G.head()

,_id,ltable_id,rtable_id,ltable_name,ltable_addr,ltable_city,ltable_phone,rtable_name,rtable_addr,rtable_city,rtable_phone,gold
1,1,549,1,the palm,9001 santa monica blvd.,los angeles,310/550-8811,apple pan the,10801 w. pico blvd.,west la,310-475-3585,0
5,5,695,1,rjs the rib joint,252 n. beverly dr.,los angeles,310/274-7427,apple pan the,10801 w. pico blvd.,west la,310-475-3585,0
27,27,983,4,the bacchanal,3570 las vegas blvd. s,las vegas,702/731-7525,belvedere the,9882 little santa monica blvd.,beverly hills,310-788-2306,0
39,39,886,8,stage deli,834 7th ave. between 53rd and 54th sts.,new york,212/245-7850,brents deli,19565 parthenia ave.,northridge,818-886-5679,0
57,57,708,10,west beach cafe,60 n. venice blvd.,los angeles,310/823-5396,bristol farms market cafe,1570 rosecrans ave. s.,pasadena,310-643-5229,0


For the purposes of this guide, we will load in a pre-labeled dataset (of 450 tuple pairs) included in this package.

In [16]:
# Load the pre-labeled data
path_G = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/lbl_restnt_wf1.csv'
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
len(G)

450

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [17]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcher\matcherutils.py:98: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  idx_values = pd.np.array(labeled_data.index.values)


## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation


### Creating a set of learning-based matchers

In [18]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [19]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\feature\attributeutils.py:191: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if returned_type == bool or returned_type == pd.np.bool_:


### Converting the development set to feature vectors

In [20]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='gold',
                            show_progress=False)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\feature\extractfeatures.py:157: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  c_splits = pd.np.array_split(candset, n_procs)


In [21]:
import os
os.getcwd()

'C:\\Users\\rmartinez4\\Box\\UCSD MAS DSE\\2019-rgm001\\DSE203\\Assignments\\Assignment 2'

In [22]:
print(dir(pickle))

NameError: name 'pickle' is not defined

In [ ]:
import cloudpickle
print(cloudpickle.__file__)

In [ ]:
import pickle
print(pickle.__file__)

In [ ]:
# Display first few rows
H.head(3)

### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [23]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric_to_select_matcher='precision', random_state=0)
result['cv_stats']

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcherselector\mlmatcherselection.py:128: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  val_list.append(pd.np.mean(scores))
c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcherselector\mlmatcherselection.py:132: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  mean_score_list.append(pd.np.mean(scores))
c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcherselector\mlmatcherselection.py:134: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if met == metric_to_select_matcher and pd.np.mean(scores) > max_score:
c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcherselec

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.976471,0.936450,0.954078
1,RF,0.975735,0.963235,0.968450
2,SVM,1.000000,0.297437,0.440952
3,LinReg,0.944444,0.920497,0.929657
4,LogReg,0.974603,0.948214,0.960212
5,NaiveBayes,0.953947,0.973214,0.962530


In [24]:
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric_to_select_matcher='recall', random_state=0)
result['cv_stats']

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcherselector\mlmatcherselection.py:128: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  val_list.append(pd.np.mean(scores))
c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcherselector\mlmatcherselection.py:132: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  mean_score_list.append(pd.np.mean(scores))
c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcherselector\mlmatcherselection.py:134: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if met == metric_to_select_matcher and pd.np.mean(scores) > max_score:
c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\matcherselec

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.976471,0.936450,0.954078
1,RF,0.975735,0.963235,0.968450
2,SVM,1.000000,0.297437,0.440952
3,LinReg,0.944444,0.920497,0.929657
4,LogReg,0.974603,0.948214,0.960212
5,NaiveBayes,0.953947,0.973214,0.962530


We observe that the best matcher (RF) is  getting us the best precision and recall. So, we select this matcher and now we can proceed on to evaluating the best matcher on the unseen data (the evaluation set).

##  Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [25]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold', show_progress=False)

c:\users\rmartinez4\anaconda3\envs\py37\lib\site-packages\py_entitymatching\feature\extractfeatures.py:157: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  c_splits = pd.np.array_split(candset, n_procs)


### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [26]:
# Train using feature vectors from I 
dt.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [29]:
# Predict on L 
predictions = dt.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [30]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)

Precision : 97.14% (34/35)
Recall : 100.0% (34/34)
F1 : 98.55%
False positives : 1 (out of 35 positive predictions)
False negatives : 0 (out of 100 negative predictions)
